# TP2 - Modeling using docplex

## 1. The `docplex` python package

`DOcplex` is a python package developped by IBM &mdash; It provides easy-to-use API for IBM solvers Cplex and Cpoptimizer.

DOcplex documentation for mathematical programming can be found here: http://ibmdecisionoptimization.github.io/docplex-doc/#mathematical-programming-modeling-for-python-using-docplex-mp-docplex-mp

## 2. Solving TSP using `docplex`

### 2.1. TSP model using `docplex`

**Exercice:** Using `docplex`, create a model for the travelling salesman problem using the MTZ or Flow formulation and compare them.

In [1]:
#!pip install cplex

In [10]:
from docplex.mp.model import Model
import tsp.data as data
import numpy as np


#distances = data.grid42
distances = data.grid17
N = len(distances)

tsp_flow = Model("TSP Flow")
tsp_flow.log_output = False

tsp_mtz = Model("TSP MTZ")
tsp_mtz.log_output = True

#Variables
x = [tsp_flow.binary_var_list(N,name=f'x{i}_') for i in range(N)]
y = [tsp_flow.integer_var_list(N,name=f'y{i}_') for i in range(N)]

t = [tsp_mtz.binary_var_list(N,name=f't{i}_') for i in range(N)]
u = tsp_mtz.integer_var_list(N,name=f'u_')

#Objective
#tsp.minimize(sum(tsp.dot(distances[i][j], x[i][j]) for i in range(N) for j in range(N)))
mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*x[i][j]
tsp_flow.minimize(mySum)

mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*t[i][j]
tsp_mtz.minimize(mySum)

#Constraints
#Constraints on variable x & t
for i in range(N) :
    tsp_flow.add_constraint(sum(x[i][j] for j in range(N) if i!=j) == 1)
    tsp_mtz.add_constraint(sum(t[i][j] for j in range(N) if i!=j) == 1)
    
for j in range(N) :
    tsp_flow.add_constraint(sum(x[i][j] for i in range(N) if i!=j) == 1)
    tsp_mtz.add_constraint(sum(t[i][j] for i in range(N) if i!=j) == 1)
    
for i in range(N):
    tsp_flow.add_constraint(x[i][i] == 0)
    tsp_mtz.add_constraint(t[i][i] == 0)

#Constraints on variable u    
tsp_mtz.add_constraint(u[0] == 1)

for i in range(1,N) : 
    tsp_mtz.add_constraint(2 <= u[i])
    tsp_mtz.add_constraint(u[i] <= N)
    tsp_mtz.add_constraints( [u[i]- u[j] + 1 <= (N-1)*(1-t[i][j]) for j in range(1,N)] )

#Constraints on variable y    
tsp_flow.add_constraint(sum(y[0][j] for j in range(N)) == 1)

for i in range(1,N):
    tsp_flow.add_constraint(sum(y[i][j] for j in range(N)) == sum(y[j][i] for j in range(N))+ 1)

for i in range(N):
    tsp_flow.add_constraints([y[i][j] <= (N * x[i][j]) for j in range(N)])


solution_flow = tsp_flow.solve()
print("Flow : z* =", solution_flow.objective_value)


#solution_mtz = tsp_mtz.solve()
#print("MTZ  : z* =", solution_mtz.objective_value)

Flow : z* = 2085.0


**On a remarqué que le tsp flow est bien plus rapide que le tsp MTZ.**

The largest set of distances contains 42 nodes, and should be easily solved by `docplex`.

### 2.2. Generating random TSP instances

**Question:** What method could you implement to generate a realistic set of distances for $n$ customers?

**Exercice:** Implement the method proposed above and test it.

In [11]:
import numpy as np
import scipy.spatial.distance as sc

def generate_distances(n: int):
    coords = np.random.rand(n, 2) #2 est le nombre de dimensions du pb
    return sc.cdist(coords, coords, metric='euclidean') #librairie qui permet d'avoir une matrice symétrique


from docplex.mp.model import Model

distances = generate_distances(20)
#distances = generate_distances(50)
print(distances)

N = len(distances)

tsp = Model("TSP")
tsp.log_output = True

# TODO: Copy your model from the first question here.
#Variables
x = [tsp.binary_var_list(N,name=f'x{i}_') for i in range(N)]
y = [tsp.integer_var_list(N,name=f'y{i}_') for i in range(N)]

#Objective
#tsp.minimize(sum(tsp.dot(distances[i][j], x[i][j]) for i in range(N) for j in range(N)))
mySum = 0
for i in range(N):
    for j in range(N):
        mySum += distances[i][j]*x[i][j]
tsp.minimize(mySum)

#Constraints
#Constraints on variable x & t
for i in range(N) :
    tsp.add_constraint(sum(x[i][j] for j in range(N) if i!=j) == 1)
    
for j in range(N) :
    tsp.add_constraint(sum(x[i][j] for i in range(N) if i!=j) == 1)
    
for i in range(N):
    tsp.add_constraint(x[i][i] == 0)

#Constraints on variable y    
tsp.add_constraint(sum(y[0][j] for j in range(N)) == 1)

for i in range(1,N):
    tsp.add_constraint(sum(y[i][j] for j in range(N)) == sum(y[j][i] for j in range(N))+ 1)

for i in range(N):
    tsp.add_constraints([y[i][j] <= (N * x[i][j]) for j in range(N)])

solution = tsp.solve()
print("z* =", solution.objective_value)

[[0.         0.5510119  0.28091347 0.23713964 0.60048994 0.3187789
  0.26305286 0.29188456 0.67651769 0.35040434 0.87825385 0.25237214
  0.85425183 0.71703852 0.86318303 0.25162329 0.80484136 0.42629758
  0.20537939 0.83762526]
 [0.5510119  0.         0.35424012 0.55224805 0.06300146 0.44349793
  0.29274377 0.43441252 0.40858236 0.41954689 0.4319661  0.38450446
  0.42795326 0.2132262  0.47519258 0.30143336 0.38092156 0.5545124
  0.57720247 0.47423166]
 [0.28091347 0.35424012 0.         0.19830431 0.38377359 0.10923452
  0.12328023 0.3885418  0.39638352 0.42336489 0.76377552 0.03413363
  0.57785672 0.56005454 0.7818257  0.13401267 0.52950213 0.55578454
  0.42420772 0.55687247]
 [0.23713964 0.55224805 0.19830431 0.         0.58162992 0.14161267
  0.28747096 0.47887745 0.52040244 0.52923972 0.95338248 0.16795473
  0.73421701 0.7556211  0.96320159 0.289192   0.68966013 0.63628245
  0.43859071 0.69270957]
 [0.60048994 0.06300146 0.38377359 0.58162992 0.         0.46323139
  0.33860612 0.496

Flow cuts applied:  25
Mixed integer rounding cuts applied:  4
Zero-half cuts applied:  13
Multi commodity flow cuts applied:  12
Lift and project cuts applied:  17
Gomory fractional cuts applied:  4

Root node processing (before b&c):
  Real time             =    0.30 sec. (142.72 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.30 sec. (142.72 ticks)
z* = 3.5357316911073893


## 3. Solving Warehouse Allocation using Benders decomposition with `docplex`

### 3.1. The warehouse problem

A company needs to supply a set of $n$ clients and needs to open new warehouses (from a
set of $m$ possible warehouses).
Opening a warehouse $j$ costs $f_j$ and supplying a client $i$ from a warehouse $j$ costs $c_{ij}$ per supply unit.
Which warehouses should be opened in order to satisfy all clients while minimizing the total cost?

### 3.2. Solving the warehouse problem with a single ILP

- $y_{j} = 1$ if and only if warehouse $j$ is opened.
- $x_{ij}$ is the fraction supplied from warehouse $j$ to customer $i$.

$
\begin{align}
  \text{min.} \quad & \sum_{i=1}^{n} \sum_{j=1}^{m} c_{ij} x_{ij} + \sum_{j=1}^{m} f_{j} y_{j} & \\
  \text{s.t.} \quad & \sum_{j=1}^{m} x_{ij} = 1, & \forall i \in\{1,\ldots,n\}\\
                    & x_{ij} \leq y_{j}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                    & y_{j} \in \left\{0,~1\right\}, & \forall j \in \left\{1,~\ldots,~m\right\}\\
                    & x_{ij} \geq 0, & \forall i \in \left\{1,~\ldots,~n\right\}, \forall j \in \left\{1,~\ldots,~m\right\}
\end{align}
$


**Exercice:** Implement the ILP model for the warehouse allocation problem and test it on the given instance.

In [12]:
from docplex.mp.model import Model

# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

wa = Model("Warehouse Allocation")
wa.log_output = True

# TODO: Model for the warehouse allocation.

#Variables
x = [wa.continuous_var_list(M,name=f'x{i}_') for i in range(N)]
y = wa.binary_var_list(M,name=f'y_')

#Objective
wa.minimize(sum(c[i][j]*x[i][j] for i in range(N) for j in range(M)) + sum(f[j]*y[j] for j in range(M)))

#Constraints
for i in range(N):
    wa.add_constraint(sum(x[i][j] for j in range(M)) == 1)

for i in range(N):
    for j in range(M):
        wa.add_constraint(x[i][j] <= y[j])
    
"""for j in range(M):
    wa.add_constraint(y[j] >= 0)
    wa.add_constraint(y[i] <= 1)"""
#inutile car y est une liste binaire
    
for i in range (N):
    for j in range(M):
        wa.add_constraint(x[i][j] >= 0)

solution = wa.solve()
print("z* =", solution.objective_value)

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 40.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 9 rows and 0 columns.
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 12 rows, 12 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.02 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 16 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound 

### 3.3. Benders' decomposition for the Warehouse Allocation problem

We are going to use Benders' decomposition to solve the Warehouse Allocation problem. 

#### Dual subproblem

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

#### Master problem

$
\begin{align*}
  \text{min.} \quad & \sum_{j=1}^{m} f_j y_j + z & \\
  \text{s.t.} \quad & z \geq \sum_{i=1}^{n}v_i^p - \sum_{i=1}^{n} \sum_{j=1}^{m} u_{ij}^p y_j, & \forall p\in l_1\\
                    & 0 \geq \sum_{i=1}^{n}v_i^r - \sum_{i=1}^{n} \sum_{j=1}^{n} u_{ij}^r y_j, & \forall r\in l_2\\
                    & y_{j} \in\{0,1\}, & \forall j\in\{1,\ldots,m\}
\end{align*}
$

**Exercice:** Implement the method `create_master_problem` that creates the initial master problem (without feasibility or optimality constraints) for the warehouse allocation problem.

<div class="alert alert-info alert-block">

You can use `print(m.export_as_lp_string())` to display a textual representation of a `docplex` model `m`.
    
</div>

In [69]:
from docplex.mp.model import Model
from docplex.mp.linear import Var
from docplex.mp.constr import AbstractConstraint
from typing import List, Sequence, Tuple


def create_master_problem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]]
) -> Tuple[Model, Var, Sequence[Var]]:
    """
    Creates the initial Benders master problem for the Warehouse Allocation problem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.

    Returns:
        A 3-tuple containing the docplex problem, the z variable and the y variables.
    """

    wa = Model("Warehouse Allocation - Benders master problem")

    #Master problem
    #variables
    u = [wa.continuous_var_list(M,name=f'u{i}_') for i in range(N)]
    v = wa.continuous_var_list(N,name=f'v_')
    y = wa.binary_var_list(M,name=f'y_')
    
    z = wa.continuous_var(name='z')
    
    #Objective
    wa.minimize(sum(f[j]*y[j] for j in range(M)) + z)
    

    return wa, z, y


# Check your method:
wa, z, y = create_master_problem(N, M, f, c)
print(wa.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: 20 y__0 + 20 y__1 + 20 y__2 + z
Subject To

Bounds
 0 <= y__0 <= 1
 0 <= y__1 <= 1
 0 <= y__2 <= 1

Binaries
 y__0 y__1 y__2
End



**Exercice:** Implement the method `add_optimality_constraints` that add optimality constraints to the Benders master problem. 

In [70]:
def add_optimality_constraint(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        wa: The Benders master problem (docplex.mp.model.Model).
        z: The z variable of the master problem.
        y: The y variables of the master problem.
        v: The optimal values for the v variables of the Benders dual subproblem.
        u: The optimal values for the u variables of the Benders dual subproblem.

    Return: The optimality constraint added.
    """
    
    optimality_constraint = wa.add_constraint((sum(v[i] for i in range(N))-
                                               sum(u[i][j]*y[j] for i in range(N) for j in range(M))) <= wa.get_var_by_name('z'))
    
    return optimality_constraint


**Exercice:** Implement the method `add_feasibility_constraints` that add feasibility constraints to the Benders master problem. 

In [42]:
def add_feasibility_constraints(
    N: int,
    M: int,
    wa: Model,
    z: Var,
    y: Sequence[Var],
    v: Sequence[float],
    u: Sequence[Sequence[float]],
) -> List[AbstractConstraint]:
    """
    Adds an optimality constraints to the given Warehouse Allocation model
    using the given optimal values from the Benders dual subproblem.

    Args:
      - N: Number of clients.
      - M: Number of warehouses.
      - wa: The Benders master problem (docplex.mp.model.Model).
      - z: The z variable of the master problem.
      - y: The y variables of the master problem.
      - v: The extreme rays for the v variables of the Benders dual subproblem.
      - u: The extreme rays for the u variables of the Benders dual subproblem.

    Returns:
        The feasibility constraint added.
    """

    feasibility_constraint = wa.add_constraint(
        (sum(v[i] for i in range(N))-sum(u[i][j]*y[j] for i in range(N) for j in range(M))) <= 0)
 
    return feasibility_constraint



**Exercice:** Implement the method `create_dual_subproblem` that, given a solution `y` of the master problem, create the corresponding Benders dual subproblem.

$
\begin{align*}
\text{max.} \quad & \sum_{i=1}^{n} v_{i} - \sum_{i=1}^{n}\sum_{j=1}^{m} \bar{y}_{j} u_{ij} & \\
\text{s.t.} \quad & v_{i} - u_{ij} \leq c_{ij}, & \forall i\in\{1,\ldots,n\},\forall j\in\{1,\ldots,m\}\\
                  & v_{i} \in\mathbb{R},\ u_{ij} \geq 0 & \forall i \in\{1,\ldots,n\}, \forall j\in\{1,\ldots,m\}
\end{align*}
$

In [71]:
from docplex.mp.model import Model


def create_dual_subproblem(
    N: int, M: int, f: Sequence[float], c: Sequence[Sequence[float]], y: Sequence[int]
) -> Tuple[Model, Sequence[Var], Sequence[Sequence[Var]]]:
    """
    Creates a Benders dual subproblem for the Warehouse Allocation problem corresponding
    to the given master solution.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        f: Array-like containing costs of opening warehouses.
        c: 2D-array like containing transport costs from client to warehouses.
        y: Values of the y variables from the Benders master problem.

    Returns:
        A 3-tuple containing the docplex problem, the v variable and the u variables.
    """

    dsp = Model("Warehouse Allocation - Benders dual subproblem")

    # We disable pre-solve to be able to retrieve a meaningful status in the main
    # algorithm:
    dsp.parameters.preprocessing.presolve.set(0)

    #dual subproblem
    #variables
    u = [dsp.continuous_var_list(M,name=f'u{i}_') for i in range(N)]
    v = dsp.continuous_var_list(N,name=f'v_')
    
    #Objective
    dsp.maximize(sum(v[i] for i in range(N)) - sum(y[j]*u[i][j] for i in range(N) for j in range(M)))
    

    #Constraints
    for i in range(N):
        for j in range(M):
            dsp.add_constraint(v[i] - u[i][j] <= c[i][j])
            dsp.add_constraint(u[i][j] >= 0 )

    return dsp, v, u


# Check your method (assuming y = [1 1 1 ... 1]):
dsp, v, u = create_dual_subproblem(N, M, f, c, [1] * M)
print(dsp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders dual subproblem

Maximize
 obj: - u0__0 - u0__1 - u0__2 - u1__0 - u1__1 - u1__2 - u2__0 - u2__1 - u2__2
      + v__0 + v__1 + v__2
Subject To
 c1: - u0__0 + v__0 <= 15
 c2: u0__0 >= 0
 c3: - u0__1 + v__0 <= 1
 c4: u0__1 >= 0
 c5: - u0__2 + v__0 <= 2
 c6: u0__2 >= 0
 c7: - u1__0 + v__1 <= 1
 c8: u1__0 >= 0
 c9: - u1__1 + v__1 <= 16
 c10: u1__1 >= 0
 c11: - u1__2 + v__1 <= 3
 c12: u1__2 >= 0
 c13: - u2__0 + v__2 <= 4
 c14: u2__0 >= 0
 c15: - u2__1 + v__2 <= 1
 c16: u2__1 >= 0
 c17: - u2__2 + v__2 <= 17
 c18: u2__2 >= 0

Bounds
End



**Exercice:** Using the methods you implemented, write the Benders decomposition algorithm for the warehouse allocation problem.

<div class="alert alert-block alert-info">

The `get_extreme_rays` function can be used to retrieve the extreme rays associated with an unbounded solution of the dual subproblem.
    
</div>

<div class="alert alert-block alert-info">
    
You can use `model.get_solve_status()` to obtain the status of the resolution and compare it to members of `JobSolveStatus`:
    
```python
if model.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION:
    pass
```
    
</div>

In [75]:
from docplex.mp.model import Model
from docplex.util.status import JobSolveStatus


def get_extreme_rays(
    N: int, M: int, model: Model, v: Sequence[Var], u: Sequence[Sequence[Var]]
) -> Tuple[Sequence[float], Sequence[Sequence[float]]]:
    """
    Retrieves the extreme rays associated to the dual subproblem.

    Args:
        N: Number of clients.
        M: Number of warehouses.
        model: The Benders dual subproblem model (docplex.mp.model.Model).
        v: 1D array containing the v variables of the subproblem.
        u: Either a 2D array of a tuple-index dictionary containing the u variables
            of the subproblem.

    Returns:
        A 2-tuple containing the list of extreme rays correspondig to v,
        and the 2D-list of extreme rays corresponding to u.
    """
    
    ray = model.get_engine().get_cplex().solution.advanced.get_ray()

    if isinstance(u, dict):

        def get_uij(i, j):
            return u[i, j]

    else:

        def get_uij(i, j):
            return u[i][j]

    return (
        [ray[v[i].index] for i in range(N)],
        [[ray[get_uij(i, j).index] for j in range(M)] for i in range(N)],
    )


# We will start with a small instances with 3 warehouses and 3 clients:
N = 3
M = 3

# Opening and distribution costs:
f = [20, 20, 20]
c = [[15, 1, 2], [1, 16, 3], [4, 1, 17]]

# We stop iterating if the new solution is less than epsilon
# better than the previous one:
epsilon = 1e-6

wa, z, y = create_master_problem(N, M, f, c)
n = 0

#on resout une première fois le master
sol_master = wa.solve()
print(wa.export_as_lp_string())

while True:

    # Print iteration:
    n = n + 1
    print("Iteration {}".format(n))
    
    #on crée le dual à l'aide des y du master et on le résout
    dsp, v, u = create_dual_subproblem(N,M,f,c,sol_master.get_values(y))
    sol_dsp = dsp.solve()
    
    
    
    print("Le statut est :", dsp.get_solve_status())
    #sinon il est faisable et on ajoute les contraintes d'optimalité
    if (dsp.get_solve_status() == JobSolveStatus.OPTIMAL_SOLUTION):
        print("1er cas : OPTIMAL")
        add_optimality_constraint(N,M,wa,sol_dsp.objective_value,y,v,u)
        
        if(abs(sol_master.get_value(z)-sol_dsp.get_value(z))<=epsilon):
            pass
        
    #si il est UNBOUNDED on doit ajouter les contraintes de faisabilité
    if (dsp.get_solve_status() == JobSolveStatus.UNBOUNDED_SOLUTION):
        print("2eme cas : UNBOUNDED")
        ray_v, ray_u = get_extreme_rays(N,M,dsp,v,u)
        add_feasibility_constraints(N,M,wa,z,y,ray_v,ray_u)
        
    sol_master = wa.solve()
    print(wa.export_as_lp_string())
    
print("Done.")

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: 20 y__0 + 20 y__1 + 20 y__2 + z
Subject To

Bounds
 0 <= y__0 <= 1
 0 <= y__1 <= 1
 0 <= y__2 <= 1

Binaries
 y__0 y__1 y__2
End

Iteration 1
Le statut est : JobSolveStatus.UNBOUNDED_SOLUTION
2eme cas : UNBOUNDED
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Warehouse Allocation - Benders master problem

Minimize
 obj: 20 y__0 + 20 y__1 + 20 y__2 + z
Subject To
 c1: - 3 y__0 - 3 y__1 - 3 y__2 <= -3

Bounds
 0 <= y__0 <= 1
 0 <= y__1 <= 1
 0 <= y__2 <= 1

Binaries
 y__0 y__1 y__2
End

Iteration 2
Le statut est : JobSolveStatus.OPTIMAL_SOLUTION
1er cas : OPTIMAL


DOcplexException: Cannot mix objects from different models in new_binary_constraint. obj1=-u0__0*y__0-u0__1*y__1-u0__2*y__2-u1__0*y__0-u1__1*y__1-u1__2*y__2-u2__0*y__0-u2__1*y__1-u2__2*y__2+v__0+v__1+v__2, obj2=z

### 3.4. Generating instances for the Warehouse Allocation problem

**Exercice:** Using the TSP instances contained in `tsp.data` or the `generate_distances` method, create instances for the warehouse allocation problem with randomized opening costs.

In [23]:
def create_instance_of_warehouse(n,max_cost):
    c = generate_distances(n)
    opening_cost = np.random.randint(max_cost)
    print(opening_cost)
    f = []
    for i in range(n):
        f.append(opening_cost)
    
    return f,c

create_instance_of_warehouse(4,50)

10


([10, 10, 10, 10], array([[0.        , 0.72976085, 0.90071523, 0.42219408],
        [0.72976085, 0.        , 0.51225776, 0.41882403],
        [0.90071523, 0.51225776, 0.        , 0.48150789],
        [0.42219408, 0.41882403, 0.48150789, 0.        ]]))

<div class="alert alert-block alert-danger"></div>